## Hi, Arsh Jaswal (230205) this side.
### __Please read readme.md file before going through the chatbot__
#### This is the sample output (Example) of the model generated in this jupyter notebook
#### You will be able to see this output at end of this notebook.
----
1. maintains history
2. used retrieved data
3. used Llama3.2 for text generation
4. local

----
```bash
Welcome to the chatbot for CP, Type 'exit' or 'quit' to terminate the conversation.

You:  It has finally been decided to build a roof over the football field in School 179. Its construction will require placing n consecutive vertical pillars. Furthermore, the headmaster wants the heights of all the pillars to form a permutation p of integers from 0 to n−1, where pi is the height of the i-th pillar from the left (1≤i≤n). As the chief, you know that the cost of construction of consecutive pillars is equal to the maximum value of the bitwise XOR of heights of all pairs of adjacent pillars. In other words, the cost of construction is equal to max1≤i≤n−1pi⊕pi+1, where ⊕ denotes the bitwise XOR operation. Find any sequence of pillar heights p of length n with the smallest construction cost. In this problem, a permutation is an array consisting of n distinct integers from 0 to n−1 in arbitrary order. For example, [2,3,1,0,4] is a permutation, but [1,0,1] is not a permutation (1 appears twice in the array) and [1,0,3] is also not a permutation (n=3, but 3 is in the array). Each test contains multiple test cases. The first line contains the number of test cases t (1≤t≤104). Description of the test cases follows. The only line for each test case contains a single integer n (2≤n≤2⋅105) — the number of pillars for the construction of the roof. It is guaranteed that the sum of n over all test cases does not exceed 2⋅105. For each test case print n integers p1, p2, …, pn — the sequence of pillar heights with the smallest construction cost. If there are multiple answers, print any of them. For n=2 there are 2 sequences of pillar heights:  For n=3 there are 6 sequences of pillar heights: 
Searching...

Bot:  Welcome to our competitive programming chat! I'll be your assistant, and we can work through problems together.

You've provided a section titled "Similar Problem," which is problem 1632B - Roof Construction. This problem involves finding the minimum cost of constructing a roof with n consecutive vertical pillars, where the cost is determined by the maximum value of the bitwise XOR of heights of adjacent pillars.

Let's break down the solution:

1. The cost of construction is a power of two.
2. The cost of construction is 2k, where k is the highest set bit in n−1.
3. There will always be a pair of adjacent elements where one of them has the k-th bit set and the other one doesn't, so the cost is at least 2k.

A simple construction that reaches this minimum cost is to have all numbers with their k-th bit set, followed by all numbers without the k-th bit set. This sequence ensures that there are no adjacent elements with different k-th bits, minimizing the cost.

Now, let's move on to the next step. Since you've asked me to continue our chat as if it were a normal conversation, I'll ask: How can I assist you further? Do you have any specific questions about this problem or would you like to discuss other topics related to competitive programming?


You:  thank you so much, It worked
Searching...

Bot:  So you're excited that the previous solution worked for you. That's great!

Now, how can I help you with the current problem you'd like to tackle? Is there a specific problem you'd like to work on together, or do you need help understanding a particular concept or technique in competitive programming?

Also, I noticed we have a conversation about problem 1632B - Roof Construction earlier. If you're still stuck on that one or would like to discuss it further, feel free to bring it up!
```
----

## Lets start

# Using Ollama's Llama3.2 for text generation.

In [2]:
from langchain.llms import Ollama
ollama = Ollama(base_url='http://localhost:11434', model="llama3.2")

C:\Users\arshc\AppData\Local\Temp\ipykernel_35744\2918043321.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama = Ollama(base_url='http://localhost:11434', model="llama3.2")


## Defining class CodeBERTEmbedder and VectorStore
----

### __CodeBERTEmbedder__ to generate embeddings from text

In [3]:
import faiss
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM

class CodeBERTEmbedder:
    def __init__(self):
        self.tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
        self.model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base")

    def generate_embedding(self, text):
        inputs = self.tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        lastHS = outputs.hidden_states[-1]

        #Attention masking
        attention_mask = inputs['attention_mask'].unsqueeze(-1)  #batch size, seq len, 1
        masked_embeddings = lastHS * attention_mask 
        pooled_embedding = masked_embeddings.sum(dim=1) / attention_mask.sum(dim=1)  # Mean pooling

        return pooled_embedding.squeeze().numpy()

    def batch_generate_embeddings(self, texts):
        inputs = self.tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        lastHS = outputs.hidden_states[-1]

        attention_mask = inputs['attention_mask'].unsqueeze(-1)
        masked_embeddings = lastHS * attention_mask
        pooled_embeddings = masked_embeddings.sum(dim=1) / attention_mask.sum(dim=1)

        return pooled_embeddings.numpy()


### __VectorStore__ class to store embeddings of problems.

##### We can also use dummy questions (to handle more broader queries) by text generation models like Llama3.2 itself.
##### Also the vector search is based on question embeddings. So if searched by solution, it won't work usually.

In [58]:
class VectorStore:
    def __init__(self, embedding_dim):
        self.index = faiss.IndexFlatL2(embedding_dim)
        self.problem_store = {}
        self.metadata_store = {}
        self.solution_store = {}
        self.prob_store = {}
        self.meta_store = {}
        self.soln_store = {}

    def add_embeddings(self, problem_id, question_embedding, metadata_embedding, answer_embedding, problem, metadata, solution):
        
        #Normalization (L2)
        question_embedding = question_embedding / np.linalg.norm(question_embedding)
        question_embedding = np.array([question_embedding], dtype=np.float32)
        self.index.add(question_embedding)

        #Storing embeddings and texts
        self.problem_store[problem_id] = question_embedding
        self.metadata_store[problem_id] = metadata_embedding
        self.solution_store[problem_id] = answer_embedding
        self.prob_store[problem_id] = problem
        self.meta_store[problem_id] = metadata
        self.soln_store[problem_id] = solution

    def search(self, query_question, top_k=2):
        #normalization
        query_question = query_question / np.linalg.norm(query_question)
        query_embedding = np.array([query_question], dtype=np.float32)
        distances, indices = self.index.search(query_embedding, top_k)

        results = []
        for idx in indices[0]:
            if idx < len(self.problem_store):
                problem_id = list(self.problem_store.keys())[idx]
                problem = self.prob_store[problem_id]
                metadata = self.meta_store[problem_id]
                solution = self.soln_store[problem_id]
                distance = distances[0][indices[0].tolist().index(idx)]
                results.append((problem_id,problem, metadata, solution, distance))

        return results


## Next task is to store the files from problems, editorials, metadata directories in some datastructure so that we can use them

In [4]:
import os
import json
os.chdir("C:/Users/arshc/ChatbotGDG/230205_Arsh_Jaswal")

In [5]:

def load_data_from_directories(problem_dir, editorial_dir, metadata_dir):
    problems = {}
    solutions = {}
    metadata = {}

    for filename in os.listdir(problem_dir):
        problem_id = os.path.splitext(filename)[0]
        with open(os.path.join(problem_dir, filename), "r", encoding="utf-8") as f:
            problems[problem_id] = f.read().strip()

    for filename in os.listdir(editorial_dir):
        problem_id = os.path.splitext(filename)[0]
        with open(os.path.join(editorial_dir, filename), "r", encoding="utf-8") as f:
            solutions[problem_id] = f.read().strip()

    for filename in os.listdir(metadata_dir):
        problem_id = os.path.splitext(filename)[0]
        with open(os.path.join(metadata_dir, filename), "r", encoding="utf-8") as f:
            metadata[problem_id] = json.load(f)

    return problems, solutions, metadata


if __name__ == "__main__":
    embedder = CodeBERTEmbedder()
    problem_dir = "data/problems"
    editorial_dir = "data/editorials"
    metadata_dir = "data/metadata"

    problems, solutions, metadata = load_data_from_directories(problem_dir, editorial_dir, metadata_dir)

    vector_store = VectorStore(embedding_dim=768)
    for problem_id in problems:
        if problem_id in solutions and problem_id in metadata:
            problem_embedding = embedder.generate_embedding(problems[problem_id])
            problem_text = problems[problem_id]
            metadata_text = ", ".join(metadata[problem_id].get("tags", []))
            metadata_embedding = embedder.generate_embedding(metadata_text)
            solution_embedding = embedder.generate_embedding(solutions[problem_id])

            vector_store.add_embeddings(
                problem_id=problem_id,
                question_embedding=problem_embedding,
                metadata_embedding=metadata_embedding,
                answer_embedding=solution_embedding,
                problem=problems[problem_id],
                metadata=metadata[problem_id],
                solution=solutions[problem_id],
            )



Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Took time of 21 min 11.7 sec to store all the files in array and from array to vectorstore map so that I can use FAISS.
----

## DEBUGGING (IGNORE)

In [6]:
query_statement="""
It has finally been decided to build a roof over the football field in School 179. Its construction will require placing n consecutive vertical pillars. Furthermore, the headmaster wants the heights of all the pillars to form a permutation p of integers from 0 to n−1, where pi is the height of the i-th pillar from the left (1≤i≤n).
As the chief, you know that the cost of construction of consecutive pillars is equal to the maximum value of the bitwise XOR of heights of all pairs of adjacent pillars. In other words, the cost of construction is equal to max1≤i≤n−1pi⊕pi+1, where ⊕ denotes the bitwise XOR operation.
Find any sequence of pillar heights p of length n with the smallest construction cost.
In this problem, a permutation is an array consisting of n distinct integers from 0 to n−1 in arbitrary order. For example, [2,3,1,0,4] is a permutation, but [1,0,1] is not a permutation (1 appears twice in the array) and [1,0,3] is also not a permutation (n=3, but 3 is in the array).
Each test contains multiple test cases. The first line contains the number of test cases t (1≤t≤104). Description of the test cases follows.
The only line for each test case contains a single integer n (2≤n≤2⋅105) — the number of pillars for the construction of the roof.
It is guaranteed that the sum of n over all test cases does not exceed 2⋅105.
For each test case print n integers p1, p2, …, pn — the sequence of pillar heights with the smallest construction cost.
If there are multiple answers, print any of them.
For n=2 there are 2 sequences of pillar heights: 
For n=3 there are 6 sequences of pillar heights: 
"""

In [7]:
query_statement=input()

In [8]:
user_query = query_statement
query_embedding = embedder.generate_embedding(user_query)
results = vector_store.search(query_question=query_embedding, top_k=3)

f=0
print("Search Results:")
for result in results:
    problem_id, problem, metadata, solution, distance = result
    print(f"Problem ID: {problem_id}")
    if problem_id=="1362A":
        f=1
    # print(f"Problem: {problem}")
    # print(f"Tags: {metadata.get('tags', [])}")
    # print(f"Solution: {solution}")
    print(f"Distance: {distance}")
    print("---------"*5)
if f==1:
    print("yes")
else:
    print("No")

Search Results:
Problem ID: 1632B
Distance: 0.005791706033051014
---------------------------------------------
Problem ID: 1920B
Distance: 0.007134918123483658
---------------------------------------------
Problem ID: 1859C
Distance: 0.007216574624180794
---------------------------------------------
No


----
## Defined a function below which takes query problem, vectorstore and embedder to find the nearest k problem.
# __Note:__
## I have set k=1, just to demonstrate, you can anytime change it to 3 or 5 or even more, according to the needs.
----
# How to choose k?
#### increasing k will increase prompt, so there are higher chance that the retrieved contains the problem asked.
##### Since prompt is now long due to higher k, it can also lead to some wrong results.

In [30]:
def search_problem_context(query_problem, vector_store=vector_store, embedder=embedder):


    query_problem_embedding = embedder.generate_embedding(query_problem)


    results = vector_store.search(
        query_question=query_problem_embedding,
        top_k=1
    )

    # Print the results
    context=""
    temp=1

    # if not results:
    #     print("NO")

    for result in results:
        # problem_id, problem, metadata, solution, distance = result
        # print(f"Problem ID: {problem_id}")
        # print(f"Problem: {problem}")
        # print(f"Tags: {metadata["tags"]}")
        # print(f"Solution: {solution}")
        # print(f"Distance: {distance}")
        context="\n".join([context,
        f"""
    Context Number {temp}
    Problem Number: {result[0]}
    
    Problem statement: {result[1]}

    Tags: {result[2]["tags"]}

    Solution: {result[3]}
        """]
                    )
        
        temp+=1
    
    return context

# A template which I will directly pass to the langchain.
### This serves as the first hand information for model.

In [46]:

template="""
You are a competitive programming assistant bot.
Below are three sections: "Query", "Similar Problem", "Chat History".

If "Query" is normal conversation, continue chat, you can also use chat history.

If "Query" is a question statement, Use "Similar Problem" below as well as your own database to answer the query.
"Similar Problem" section has same or similar problem statement to query. And also has its OFFICIAL solution.
If it's similar or same, return this solution or answer.



Query: 
{user_input}



Similar Problem:

{Similar_Problem}



Chat History: {history}

"""


### The model maintains chat history. If the chat is too long, it may lead to some misleading answers due to context limit or token limit exceeded.
### Prefer clearing history if it nothing to do with previous problem by rerunning the Chatbot implementation cell.

-----
## Debugging again (ignore)

In [32]:
cont=search_problem_context("""
It has finally been decided to build a roof over the football field in School 179. Its construction will require placing n consecutive vertical pillars. Furthermore, the headmaster wants the heights of all the pillars to form a permutation p of integers from 0 to n−1, where pi is the height of the i-th pillar from the left (1≤i≤n).
As the chief, you know that the cost of construction of consecutive pillars is equal to the maximum value of the bitwise XOR of heights of all pairs of adjacent pillars. In other words, the cost of construction is equal to max1≤i≤n−1pi⊕pi+1, where ⊕ denotes the bitwise XOR operation.
Find any sequence of pillar heights p of length n with the smallest construction cost.
In this problem, a permutation is an array consisting of n distinct integers from 0 to n−1 in arbitrary order. For example, [2,3,1,0,4] is a permutation, but [1,0,1] is not a permutation (1 appears twice in the array) and [1,0,3] is also not a permutation (n=3, but 3 is in the array).
Each test contains multiple test cases. The first line contains the number of test cases t (1≤t≤104). Description of the test cases follows.
The only line for each test case contains a single integer n (2≤n≤2⋅105) — the number of pillars for the construction of the roof.
It is guaranteed that the sum of n over all test cases does not exceed 2⋅105.
For each test case print n integers p1, p2, …, pn — the sequence of pillar heights with the smallest construction cost.
If there are multiple answers, print any of them.
For n=2 there are 2 sequences of pillar heights: 
For n=3 there are 6 sequences of pillar heights: 
""")

print(cont)
if "1632" in cont:
    print("YES")
if "1682" in cont:
    print("YES")
if "2037" in cont:
    print("YES")



    Context Number 1
    Problem Number: 1632B
    
    Problem statement: It has finally been decided to build a roof over the football field in School 179. Its construction will require placing n consecutive vertical pillars. Furthermore, the headmaster wants the heights of all the pillars to form a permutation p of integers from 0 to n−1, where pi is the height of the i-th pillar from the left (1≤i≤n).
As the chief, you know that the cost of construction of consecutive pillars is equal to the maximum value of the bitwise XOR of heights of all pairs of adjacent pillars. In other words, the cost of construction is equal to max1≤i≤n−1pi⊕pi+1, where ⊕ denotes the bitwise XOR operation.
Find any sequence of pillar heights p of length n with the smallest construction cost.
In this problem, a permutation is an array consisting of n distinct integers from 0 to n−1 in arbitrary order. For example, [2,3,1,0,4] is a permutation, but [1,0,1] is not a permutation (1 appears twice in the array)

-----
# ChatBot for CP:

In [59]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate


# from IPython.display import display, clear_output

model=OllamaLLM(model="llama3.2")

prompt=ChatPromptTemplate.from_template(template)

chain=prompt | model

def handle_convo():
    history=""
    print("Welcome to the chatbot for CP, Type 'exit' or 'quit' to terminate the conversation. Keep conversation relevant.\nIf you are asking a question, Just copy paste the whole problem statement and Input section from codeforces site.")
    while True:
        print("\n\n")
        user_input=input("You: ")
        if user_input.lower() in ["exit","quit"]:
            break
        print("You: ",user_input)
        print("Searching...")
        context=search_problem_context(user_input)
        # print(context)
        out=chain.invoke({"history":history,"user_input":user_input,"Similar_Problem":context})
        
        # clear_output(wait=True)
        print("\n")
        print("Bot: ", out)

        history += f"\nUser: {user_input}\nAI: {out}"
        # display(f"User: {user_input}\nAI: {out}")

# Run chatbot here:
# __NOTE__:
When asking direct question to model, paste whole statement, "Input", "Output" section from codeforces page.

In [52]:

if __name__ == "__main__":
    handle_convo()

Welcome to the chatbot for CP, Type 'exit' or 'quit' to terminate the conversation.
You:  It has finally been decided to build a roof over the football field in School 179. Its construction will require placing n consecutive vertical pillars. Furthermore, the headmaster wants the heights of all the pillars to form a permutation p of integers from 0 to n−1, where pi is the height of the i-th pillar from the left (1≤i≤n). As the chief, you know that the cost of construction of consecutive pillars is equal to the maximum value of the bitwise XOR of heights of all pairs of adjacent pillars. In other words, the cost of construction is equal to max1≤i≤n−1pi⊕pi+1, where ⊕ denotes the bitwise XOR operation. Find any sequence of pillar heights p of length n with the smallest construction cost. In this problem, a permutation is an array consisting of n distinct integers from 0 to n−1 in arbitrary order. For example, [2,3,1,0,4] is a permutation, but [1,0,1] is not a permutation (1 appears twice 

### As you can see, I asked him about the problem statement and input and it guessed it is from __1632B__ (which is correct).
It also handles case where we only send conversational messages
_______
```bash

You:  thank you so much, It worked
Searching...
Bot:  So you're excited that the previous solution worked for you. That's great!

Now, how can I help you with the current problem you'd like to tackle? Is there a specific problem you'd like to work on together, or do you need help understanding a particular concept or technique in competitive programming?

Also, I noticed we have a conversation about problem 1632B - Roof Construction earlier. If you're still stuck on that one or would like to discuss it further, feel free to bring it up!
```
_____

Also, The model is remembering the past conversation and it can be confirmed from the message:
``` bash
Also, I noticed we have a conversation about problem 1632B - Roof Construction earlier. If you're still stuck on that one or would like to discuss it further, feel free to bring it up!
```
We can see it talked about problem 1632B in next query (= "thank you so much, It worked")

____

# __THANKYOU__ :)